In [1]:
import os
import pandas as pd
import json
import datetime
from tqdm import tqdm

In [2]:
#path = "Files/Authors"
path = 'Files/Authors_lates'
filePath = path + "/"
authorFiles = os.listdir(path)

#df_presenters = pd.read_pickle("Files/Dataframes/researchers_unique.pkl")  
df_researchers = pd.read_pickle("Files/Dataframes/researchers_all.pkl")
#authorID = pd.read_pickle("Files/Dictionary/authorID.pkl")  

In [3]:
def readFile(file, path): #function to read file
    with open(path + file) as f:
        data = json.load(f)
    return data

In [4]:
def getName(data): #get namme from json
    name = data['File']['data'][0]['name']#only save first name
        
    return name

In [5]:
def getFields(data): #get field
    dataCount = len(data['data']) #number of results(with authors) #number of results(with authors)
    field = None
    for i in range(0, dataCount):
        paper_data = data['data'][i]['papers']
        n_papers = len(paper_data)
        if field == 'Physics':
            break
        for p in range(0, n_papers):
            if paper_data[p]['fieldsOfStudy']:
                if 'Physics' in paper_data[p]['fieldsOfStudy']:
                    field = 'Physics'
                    break
                else:
                    field = 'other'
            else:
                field = 'unknown'
    return field

In [6]:
def getPublicationData(data): #get dict with number of publications for each year
    publishCount = {}
    dataCount = len(data['data']) #number of results(with authors)
    
    for i in range(0, dataCount):
        paper_data = data['data'][i]['papers']
        n_papers = len(paper_data)
        for p in range(0, n_papers):
            year = paper_data[p]['year']
            if year in publishCount:
                publishCount[year]+= 1 #add 1 to count
            else:
                publishCount[year] = 1 #create year, as set count as 1
                
    return publishCount

In [7]:
def getCumulativePaperCount(confYear, ID, publicationDict):
    publishCount = 0
    publications = publicationDict[ID]
    for key in publications:
        if key != None:
            if key < confYear :
                publishCount += publications[key]
            
    return publishCount

In [11]:
path = 'Files/Authors_lates/'
files = os.listdir(path)

def readFile(file, path): #function to read file
    with open(path + file) as f:
        data = json.load(f)
    return data

In [13]:
#test
keycount = 0
for file in tqdm(files):
    if file != '.DS_Store':
        data = readFile(file, path)
        for key in data.keys():
            researcher_data = data[key]
            if 'message' in data[key]['File']:
                error = 1
            else:
                if researcher_data['Name']: #if any name
                    if researcher_data['File']['data']: #if any info about researcher
                        keycount += 1

100%|█████████████████████████████████████████| 228/228 [10:20<00:00,  2.72s/it]


In [8]:
def getAuthorInfo(files, path): #get info about authors
    error = []
    researcher_dict = {}
    publicationData = {}
    for file in tqdm(files):
        if file != '.DS_Store':
            data = readFile(file, path)
            for key in data.keys():
                researcher_data = data[key]
                if 'message' in data[key]['File']:
                    error.append(researcher_data['Name'])
                else:
                    if researcher_data['Name']: #if any name
                        if researcher_data['File']['data']: #if any info about researcher
                            originalName = researcher_data['Name']
                            name = getName(researcher_data)
                            dataCount = data[key]['File']['total'] #authorResults
                        #papers = data[key]['File']['data'][0]['papers']
                        #if papers: #only if info about any papers
                            publicationData[key] = getPublicationData(data[key]['File'])
                            if not len(publicationData[key]) == 1 and publicationData[key].get(None):
#                        if not len(publicationData[key]) == 1 and publicationData[key].get(None) == 1:
                                maxPaperCount = sum(publicationData[key].values())
                                field = getFields(data[key]['File']) #get field(physics, other or unknown 
                                firstYear = min(list(filter(None,publicationData[key].keys())))
                        
                                researcher_dict[key] = {'OriginalName': originalName, 'API_Name': name, 'AuthorCount': dataCount, 'MaxPaperCount': maxPaperCount, 'Field': field, 'FirstPaperYear': firstYear}
    return error, publicationData, researcher_dict

In [9]:
def getFirstName(df):
    firstName_dict = {}
    for i in df.index:
        ID = df.iloc[i]['ID']
        name1 = df.OriginalName.iloc[i].split()
        name2 = df.API_Name.iloc[i].split()
        if name1: #if there is any originalName
            firstName = max(name1[0], name2[0], key = len) #take longest first name of the 2
            firstName_dict[ID] = firstName
        
    return firstName_dict



#### Add additional information

In [11]:
#authorFiles

In [1]:
#test
#error in 198482
data = readFile('199000.txt', filePath)
#researcher_data = data['198482']


NameError: name 'readFile' is not defined

In [13]:
#researcher_data
#['File']['data']

In [14]:
errorNames, publicationData, researcherInfo = getAuthorInfo(authorFiles, filePath) #dict with researcher info

NameError: name 'getAuthorInfo' is not defined

In [11]:
#create df from researcherInfo
researcher_df = pd.DataFrame.from_dict(researcherInfo, orient = 'index').reset_index().rename(columns = {'index': 'ID'})

In [12]:
researchers = pd.merge(researcher_df, df_researchers,  left_on=['OriginalName'], right_on=['Name']) #add information

In [13]:
firstNameDict = getFirstName(researcher_df)

In [14]:
#make firstNameDict to df
firstName_df = pd.DataFrame.from_dict(firstNameDict, orient = 'index').reset_index().rename(columns = {'index': 'ID', 0: 'FirstName'})

In [15]:
#add first name
apiResearchers = pd.merge(researchers, firstName_df,  left_on=['ID'], right_on=['ID'])

In [16]:
#add seniority
apiResearchers['Seniority'] = apiResearchers.Year - apiResearchers.FirstPaperYear 

In [17]:
#apply function to calculate cumulative paper count for current conference year
apiResearchers['CumulativePaperCount'] = apiResearchers.apply(lambda x: getCumulativePaperCount(x.Year, x.ID, publicationData), axis = 1)

In [18]:
#add productivity (cumulative papercount / years in the field)
apiResearchers['Productivity'] = apiResearchers.CumulativePaperCount/apiResearchers.Seniority

In [19]:
#select only researchers where first name does not contain "."
researchers_realNames = apiResearchers[~apiResearchers.FirstName.str.contains('\.')]

In [20]:
researchers_realNames.to_pickle("Files/DataFrames/apiResearchers.pkl") 
#now add gender